In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import optuna

from common import EP

import types

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [7]:
# mytrial = []
mytrial = list(pd.read_pickle('../trial/svr.pkl').T.to_dict().values())
df_trial = pd.DataFrame(mytrial)
len(mytrial)

1

In [8]:
df_trial.loc[0]['param']

{'algorithm': {'cls': 'SVR',
  'fit': {},
  'init': {'kernel': 'rbf',
   'degree': 2,
   'gamma': 'auto',
   'coef0': 0.0,
   'tol': 0.001,
   'C': 0.2,
   'epsilon': 0.1,
   'shrinking': True}},
 'columns': ["change_quantiles{'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'spkt_welch_density__coeff_65',
  'q05_roll_std_10',
  'Hilbert_mean_2',
  'min__roll_std',
  'spkt_welch_density__coeff_4',
  'abs_max_6',
  'spkt_welch_density__coeff_99',
  'abs_q01_7',
  'abs_max_8',
  'kurt_7',
  'partial_autocorrelationlag_5',
  'q05_2',
  'abs_max_roll_mean_100',
  'spkt_welch_density__coeff_50',
  'abs_q01_6',
  'ave10_6',
  "binned_entropy{'max_bins': 10}",
  'min_roll_std_100',
  'spkt_welch_density__coeff_113',
  'med_6',
  'spkt_welch_density__coeff_42',
  '4th_peak_psd',
  '5000min_quantile05',
  'fft_coefficientcoeff_36__attr_"abs"',
  'spkt_welch_density__coeff_41',
  'q01_roll_std_100',
  'max_to_min',
  'spkt_welch_density__coeff_17',
  'abs_min_5',
  'q25_roll_std_100',
 

In [32]:
param = {'algorithm': {'cls': 'SVR',
  'fit': {},
  'init': {'kernel': 'rbf',
   'degree': 2,
   'gamma': 'auto',
   'coef0': 0.0,
   'tol': 0.001,
   'C': 0.01,
   'epsilon': 0.01,
   'shrinking': True}},
 'columns': ['q25_roll_std_100',
 'abs_q25_5',
 'iqr_6',
 'abs_q01_4',
 'abs_q75_7',
 'spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2'],

 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},#stratified
 'scaler': {'cls': 'StandardScaler'}}

In [33]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial)

In [34]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial['C']=df_trial['param'].apply(lambda x: x['algorithm']['init']['C'])
df_trial[['datetime','kfold-type','C', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,kfold-type,C,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-15 01:34:57.305051,stratified,0.20,165,1.858539,0.000008,1.896976,0.000376,0.038436
1,2019-05-16 22:16:24.216619,group,0.20,165,1.811659,0.000800,2.165044,0.001603,0.353384
2,2019-05-16 23:02:44.635848,group,0.20,165,1.809677,0.000808,2.163866,0.001640,0.354189
3,2019-05-16 23:39:48.863510,group,0.20,165,1.821035,0.011366,2.215748,0.203167,0.394714
4,2019-05-16 23:43:14.526709,group,0.20,7,1.988295,0.002298,2.097956,0.016376,0.109661
5,2019-05-16 23:48:48.282372,group,0.05,7,2.014279,0.002717,2.102354,0.017417,0.088075
6,2019-05-16 23:52:31.919779,group,0.01,7,2.050315,0.003073,2.092530,0.015488,0.042216
7,2019-05-16 23:57:06.120463,group,0.01,7,2.050366,0.003073,2.092532,0.015490,0.042166


In [43]:
df_test_pred = df_trial.loc[342]['df_test_pred']

In [44]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission.csv', index=False)

In [35]:
df_trial.to_pickle('../trial/svr.pkl')